In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# CSV dosyasını oku
import pandas as pd
csv_file_path = '/content/drive/MyDrive/oflazer/elif/data/sikayetvar_comments_pttcell_page1_to_59.csv'
data = pd.read_csv(csv_file_path)
data = data['comment']
data

0      PTT var. Benim iznim olmadan 13.12.2023 Pazart...
1      Dün sabah telefonuma 4.5G uyumlu sim kart aldı...
2      Pttcell 6 aydır çözülemeyen çekmeme problemi ü...
3      Pttcell faturalı hattımın taahhüt bittiği için...
4      Pttcell hattında gümüş 10 GB tarifesine geçmek...
                             ...                        
958    Pttcell internet hızı sorunu, çağ dışı interne...
959    Dişçi koltuğunda protez ölçüsü alınırken Pttce...
960    Yeni taşındığım evimde cep telefon görüşmeleri...
961    Pttcell faturasız hat kullanmaktayım. 30 Hazir...
962    Pttcell müşteri hizmetlerini arayarak Bulgaris...
Name: comment, Length: 963, dtype: object

In [5]:
!pip install pandas transformers nltk

In [6]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Gerekli NLTK veri setlerini indir
nltk.download('stopwords')
nltk.download('punkt')

from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer

def preprocess_text(text):
    # Noktalama işaretlerini kaldırma
    text = re.sub(r'[^\w\s]', '', text)

    # Küçük harfe dönüştürme
    text = text.lower()

    # Sayıları çıkarma
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\b(\d{1,2})\s+(Ocak|Şubat|Mart|Nisan|Mayıs|Haziran|Temmuz|Ağustos|Eylül|Ekim|Kasım|Aralık)\s+(\d{4})\b', '', text)
    text = re.sub(r'\b(\d{1,2})/(\d{1,2})/(\d{4})\b', '', text) #denicez !!!

    # Kelime tokenizasyonu
    kelimeler = word_tokenize(text)

    # Stop words çıkarma
    stop_words = set(stopwords.words('turkish'))
    kelimeler_filtreli = [kelime for kelime in kelimeler if kelime not in stop_words]

    # Her kelimeden sadece bir tanesini içeren set
    unique_kelimeler = list(set(kelimeler_filtreli))

    # Filtrelenmiş ve tekilleştirilmiş kelimeleri birleştirip metin olarak döndürme
    text = ' '.join(unique_kelimeler)

    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:

from collections import Counter
# NER modeli ve tokenizer yükleme
model = AutoModelForTokenClassification.from_pretrained("savasy/bert-base-turkish-ner-cased")
tokenizer = AutoTokenizer.from_pretrained("savasy/bert-base-turkish-ner-cased")
ner = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="first")


# Veri ön işleme
processed_data = [preprocess_text(comment) for comment in data]

# Tüm metinleri birleştir
all_text = ' '.join(processed_data)

# Kelimeleri tokenize et
all_tokens = word_tokenize(all_text)

# Kelime frekanslarını hesapla
word_freq = Counter(all_tokens)

# En sık geçen kelimeleri belirle (ilk 10 kelime)
most_common_words = [word for word, freq in word_freq.most_common(100)]

# Boş bir DataFrame oluştur
df = pd.DataFrame(columns=["comment", "entity_list"])

Some weights of the model checkpoint at savasy/bert-base-turkish-ner-cased were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [8]:
most_common_words

['bir',
 'pttcell',
 'müşteri',
 'tl',
 'fatura',
 'bana',
 'ay',
 'gb',
 'olarak',
 'ben',
 'hizmetlerini',
 'yok',
 'istiyorum',
 'internet',
 'rağmen',
 'taahhüt',
 'sonra',
 'hat',
 'gün',
 'ancak',
 'ptt',
 'var',
 'tarife',
 'olan',
 'başka',
 'yeni',
 'kadar',
 'aradım',
 'hiçbir',
 'önce',
 'paket',
 'hizmetleri',
 'böyle',
 'ediyorum',
 'talep',
 'geldi',
 'tarihinde',
 'fakat',
 'aynı',
 'hattımı',
 'şekilde',
 'benim',
 'mesaj',
 'türk',
 'aldım',
 'fazla',
 'olduğunu',
 'kabul',
 'iptal',
 'faturalı',
 'aradığımda',
 'yaptım',
 'faturam',
 'tekrar',
 'son',
 'bilgi',
 'söylediler',
 'telefon',
 'değil',
 'bile',
 'geçiş',
 'telekom',
 'an',
 'çözüm',
 'hattım',
 'beni',
 'paketi',
 'ilgili',
 'ücret',
 'olduğu',
 'mobil',
 'sürekli',
 'halde',
 'oldu',
 'bugün',
 'numaralı',
 'dakika',
 'pttcelle',
 'herhangi',
 'hala',
 'aylık',
 'mağdur',
 'bunun',
 'kullanıyorum',
 'saat',
 'lira',
 'yüksek',
 'ettim',
 'üzerinden',
 'sadece',
 'bedeli',
 'sms',
 'şikayet',
 'içinde',
 '

In [9]:


for comment in data:
    # Yorumları ön işleme
    processed_comment = preprocess_text(comment)

    # Sadece en sık geçen kelimelerle sınırlı metin oluşturma
    filtered_comment = ' '.join([word for word in word_tokenize(processed_comment) if word in most_common_words])

    # Entity tespiti
    entities = ner(filtered_comment)

    # Sadece "ORG" etiketli entitileri filtrele
    org_entities = [entity['word'] for entity in entities if entity['entity_group'] == 'ORG']

    # Yeni satırı DataFrame'e ekle
    new_row = {"comment": comment, "entity_list": org_entities}
    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)


In [11]:
df

,comment,entity_list
0,PTT var. Benim iznim olmadan 13.12.2023 Pazart...,"[ptt, ilgili]"
1,Dün sabah telefonuma 4.5G uyumlu sim kart aldı...,[]
2,Pttcell 6 aydır çözülemeyen çekmeme problemi ü...,[pttcell]
3,Pttcell faturalı hattımın taahhüt bittiği için...,[pttcell]
4,Pttcell hattında gümüş 10 GB tarifesine geçmek...,[pttcell]
...,...,...
958,"Pttcell internet hızı sorunu, çağ dışı interne...",[pttcell internet]
959,Dişçi koltuğunda protez ölçüsü alınırken Pttce...,[pttcell]
960,Yeni taşındığım evimde cep telefon görüşmeleri...,[]
961,Pttcell faturasız hat kullanmaktayım. 30 Hazir...,[pttcell internet]


In [10]:

# DataFrame'i CSV olarak kaydet
df.to_csv("output.csv", index=False)

print("Veriler başarıyla işlenip 'output.csv' dosyasına kaydedildi.")


Veriler başarıyla işlenip 'output.csv' dosyasına kaydedildi.


In [12]:
import re
import pandas as pd
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer

# NLTK veri setlerini indirmek için bağımsız bir blok kullanın
nltk.download('stopwords')
nltk.download('punkt')

def preprocess_text(text):
    # Noktalama işaretlerini kaldırma
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\b(\d{1,2})\s+(Ocak|Şubat|Mart|Nisan|Mayıs|Haziran|Temmuz|Ağustos|Eylül|Ekim|Kasım|Aralık)\s+(\d{4})\b', '', text)
    text = re.sub(r'\b(\d{1,2})/(\d{1,2})/(\d{4})\b', '', text) #denicez !!!
    # Küçük harfe dönüştürme
    text = text.lower()

    # Sayıları çıkarma
    text = re.sub(r'\d+', '', text)

    # Kelime tokenizasyonu
    kelimeler = word_tokenize(text)

    # Stop words çıkarma
    stop_words = set(stopwords.words('turkish'))
    kelimeler_filtreli = [kelime for kelime in kelimeler if kelime not in stop_words]

    return kelimeler_filtreli

# NER modeli ve tokenizer yükleme
model = AutoModelForTokenClassification.from_pretrained("savasy/bert-base-turkish-ner-cased")
tokenizer = AutoTokenizer.from_pretrained("savasy/bert-base-turkish-ner-cased")
ner = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="first")

# Örnek veri

# Veri ön işleme
processed_data = [preprocess_text(comment) for comment in data]

# Tüm metinleri birleştir
all_tokens = [token for sublist in processed_data for token in sublist]

# Bigramları bulma
bigram_measures = BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(all_tokens)

# En sık geçen bigramları belirle (ilk 10 bigram)
most_common_bigrams = finder.ngram_fd.most_common(10)

print("En sık geçen bigramlar:")
for bigram, freq in most_common_bigrams:
    print(f"{bigram}: {freq}")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Some weights of the model checkpoint at savasy/bert-base-turkish-ner-cased were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is pa

En sık geçen bigramlar:
('müşteri', 'hizmetlerini'): 201
('müşteri', 'hizmetleri'): 134
('türk', 'telekom'): 105
('böyle', 'bir'): 90
('başka', 'bir'): 81
('hizmetlerini', 'aradım'): 75
('herhangi', 'bir'): 74
('pttcell', 'müşteri'): 71
('talep', 'ediyorum'): 71
('gb', 'internet'): 68


In [13]:
import re
import pandas as pd
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.collocations import TrigramAssocMeasures, TrigramCollocationFinder
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer

# NLTK veri setlerini indirmek için bağımsız bir blok kullanın
nltk.download('stopwords')
nltk.download('punkt')

def preprocess_text(text):
    # Noktalama işaretlerini kaldırma
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\b(\d{1,2})\s+(Ocak|Şubat|Mart|Nisan|Mayıs|Haziran|Temmuz|Ağustos|Eylül|Ekim|Kasım|Aralık)\s+(\d{4})\b', '', text)
    text = re.sub(r'\b(\d{1,2})/(\d{1,2})/(\d{4})\b', '', text) #denicez !!!
    # Küçük harfe dönüştürme
    text = text.lower()

    # Sayıları çıkarma
    text = re.sub(r'\d+', '', text)

    # Kelime tokenizasyonu
    kelimeler = word_tokenize(text)

    # Stop words çıkarma
    stop_words = set(stopwords.words('turkish'))
    kelimeler_filtreli = [kelime for kelime in kelimeler if kelime not in stop_words]

    return kelimeler_filtreli

# NER modeli ve tokenizer yükleme
model = AutoModelForTokenClassification.from_pretrained("savasy/bert-base-turkish-ner-cased")
tokenizer = AutoTokenizer.from_pretrained("savasy/bert-base-turkish-ner-cased")
ner = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="first")

# Veri ön işleme
processed_data = [preprocess_text(comment) for comment in data]

# Tüm metinleri birleştir
all_tokens = [token for sublist in processed_data for token in sublist]

# Trigramları bulma
trigram_measures = TrigramAssocMeasures()
finder = TrigramCollocationFinder.from_words(all_tokens)

# En sık geçen trigramları belirle (ilk 10 trigram)
most_common_trigrams = finder.ngram_fd.most_common(20)

print("En sık geçen trigramlar:")
for trigram, freq in most_common_trigrams:
    print(f"{trigram}: {freq}")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Some weights of the model checkpoint at savasy/bert-base-turkish-ner-cased were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is pa

En sık geçen trigramlar:
('müşteri', 'hizmetlerini', 'aradım'): 74
('müşteri', 'hizmetlerini', 'aradığımda'): 44
('bir', 'an', 'önce'): 33
('pttcell', 'müşteri', 'hizmetlerini'): 29
('başka', 'bir', 'operatöre'): 25
('müşteri', 'hizmetlerini', 'arayarak'): 20
('pttcell', 'faturalı', 'hat'): 19
('pttcell', 'müşteri', 'hizmetleri'): 17
('tüketici', 'hakem', 'heyetine'): 16
('iadesini', 'talep', 'ediyorum'): 16
('müşteri', 'hizmetlerini', 'arıyorum'): 15
('türk', 'telekom', 'pttcell'): 15
('talep', 'ediyorum', 'pttcell'): 14
('mağduriyetimin', 'giderilmesini', 'istiyorum'): 13
('tl', 'fatura', 'geldi'): 11
('pttcell', 'türk', 'telekom'): 11
('müşteri', 'hizmetlerine', 'ulaşamıyorum'): 11
('gb', 'internet', 'dakika'): 10
('pttcell', 'dopdolu', 'gb'): 10
('pttcell', 'müşteri', 'hizmetlerinden'): 10


In [14]:

# Boş bir DataFrame oluştur
df = pd.DataFrame(columns=["comment", "entity_list"])

for comment in data:
    # Yorumları ön işleme
    processed_comment = preprocess_text(comment)

    # Sadece en sık geçen trigramlara göre metin oluşturma
    filtered_comment = ' '.join([' '.join(trigram) for trigram in nltk.trigrams(processed_comment) if trigram in [t[0] for t in most_common_trigrams]])

    # Entity tespiti
    entities = ner(filtered_comment)

    # Sadece "ORG" etiketli entitileri filtrele
    org_entities = [entity['word'] for entity in entities if entity['entity_group'] == 'ORG']

    # Yeni satırı DataFrame'e ekle
    new_row = {"comment": comment, "entity_list": org_entities}
    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)


In [15]:
df

,comment,entity_list
0,PTT var. Benim iznim olmadan 13.12.2023 Pazart...,[müşteri]
1,Dün sabah telefonuma 4.5G uyumlu sim kart aldı...,"[müşteri hizmetlerini, müşteri]"
2,Pttcell 6 aydır çözülemeyen çekmeme problemi ü...,[]
3,Pttcell faturalı hattımın taahhüt bittiği için...,[]
4,Pttcell hattında gümüş 10 GB tarifesine geçmek...,[]
...,...,...
958,"Pttcell internet hızı sorunu, çağ dışı interne...",[]
959,Dişçi koltuğunda protez ölçüsü alınırken Pttce...,[]
960,Yeni taşındığım evimde cep telefon görüşmeleri...,[]
961,Pttcell faturasız hat kullanmaktayım. 30 Hazir...,[]


In [16]:
import re
import pandas as pd
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder, TrigramAssocMeasures, TrigramCollocationFinder
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer

# NLTK veri setlerini indirmek için bağımsız bir blok kullanın
nltk.download('stopwords')
nltk.download('punkt')

def preprocess_text(text):
    # Noktalama işaretlerini kaldırma
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\b(\d{1,2})\s+(Ocak|Şubat|Mart|Nisan|Mayıs|Haziran|Temmuz|Ağustos|Eylül|Ekim|Kasım|Aralık)\s+(\d{4})\b', '', text)
    text = re.sub(r'\b(\d{1,2})/(\d{1,2})/(\d{4})\b', '', text) #denicez !!!
    # Küçük harfe dönüştürme
    text = text.lower()

    # Sayıları çıkarma
    text = re.sub(r'\d+', '', text)

    # Kelime tokenizasyonu
    kelimeler = word_tokenize(text)

    # Stop words çıkarma
    stop_words = set(stopwords.words('turkish'))
    kelimeler_filtreli = [kelime for kelime in kelimeler if kelime not in stop_words]

    return kelimeler_filtreli

# NER modeli ve tokenizer yükleme
model = AutoModelForTokenClassification.from_pretrained("savasy/bert-base-turkish-ner-cased")
tokenizer = AutoTokenizer.from_pretrained("savasy/bert-base-turkish-ner-cased")
ner = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="first")

# Veri ön işleme
processed_data = [preprocess_text(comment) for comment in data]

# Tüm metinleri birleştir
all_tokens = [token for sublist in processed_data for token in sublist]

# Tekil kelimeleri bulma
word_freq = Counter(all_tokens)
most_common_words = [word for word, freq in word_freq.most_common(10)]

# Bigramları bulma
bigram_measures = BigramAssocMeasures()
bigram_finder = BigramCollocationFinder.from_words(all_tokens)
most_common_bigrams = bigram_finder.ngram_fd.most_common(10)

# Trigramları bulma
trigram_measures = TrigramAssocMeasures()
trigram_finder = TrigramCollocationFinder.from_words(all_tokens)
most_common_trigrams = trigram_finder.ngram_fd.most_common(10)

# Boş bir DataFrame oluştur
df = pd.DataFrame(columns=["comment", "entity_list"])

for comment in data:
    # Yorumları ön işleme
    processed_comment = preprocess_text(comment)

    # NER ile entity tespiti
    entities = ner(comment)
    org_entities = [entity['word'] for entity in entities if entity['entity_group'] == 'ORG']

    # En sık geçen kelimeler, bigramlar ve trigramlar
    words_in_comment = [word for word in most_common_words if word in processed_comment]
    bigrams_in_comment = [' '.join(bigram) for bigram, freq in most_common_bigrams if all(word in processed_comment for word in bigram)]
    trigrams_in_comment = [' '.join(trigram) for trigram, freq in most_common_trigrams if all(word in processed_comment for word in trigram)]

    # Entity listesine ekleme
    all_entities = list(set(org_entities + words_in_comment + bigrams_in_comment + trigrams_in_comment))

    # Yeni satırı DataFrame'e ekle
    new_row = {"comment": comment, "entity_list": all_entities}
    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Some weights of the model checkpoint at savasy/bert-base-turkish-ner-cased were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is pa

In [23]:
new_row

{'comment': "Pttcell müşteri hizmetlerini arayarak Bulgaristan'a gideceğimi belirttim. Müşteri temsilcisi bana yurt dışı kullanımı açarak Bulgaristan'da m-cell adlı telekomünikasyon şirketi üzerinden paketinin aktif hale geleceğini belirtti.",
 'entity_list': ['müşteri',
  'müşteri hizmetlerini',
  'müşteri hizmetlerini arayarak',
  'bana',
  'yurt dışı',
  'Pttcell',
  'pttcell',
  'hizmetlerini',
  'müşteri temsilcisi',
  'pttcell müşteri',
  'pttcell müşteri hizmetlerini']}

In [24]:
from transformers import AutoModelForTokenClassification, AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch
import json

# Türkçe NER modeli ve tokenizer'ı yükleme
ner_model = AutoModelForTokenClassification.from_pretrained("savasy/bert-base-turkish-ner-cased")
ner_tokenizer = AutoTokenizer.from_pretrained("savasy/bert-base-turkish-ner-cased")


# Türkçe sentiment analiz modeli ve tokenizer'ı yükleme
sentiment_model = AutoModelForSequenceClassification.from_pretrained("emre/turkish-sentiment-analysis")
sentiment_tokenizer = AutoTokenizer.from_pretrained("emre/turkish-sentiment-analysis")

# NER pipeline oluşturma
ner = pipeline("ner", model=ner_model, tokenizer=ner_tokenizer, aggregation_strategy="first")

# Cümleyi analiz etme
sentence = """Fiber 100mb SuperOnline kullanıcısıyım yaklaşık 2 haftadır @Twitch @Kick_Turkey gibi canlı yayın platformlarında 360p yayın izlerken donmalar yaşıyoruz. Başka hiç bir operatörler bu sorunu yaşamazken ben parasını verip alamadığım hizmeti neden ödeyeyim ?"""

# Entity'leri bulma
ner_results = ner(sentence)

# Entity'leri birleştirme ve unique hale getirme
entities = list(set([result['word'] for result in ner_results]))

# Sonuçları tutmak için liste
results = []

# Her entity için analiz yapma
for entity in entities:
    inputs = sentiment_tokenizer(entity, return_tensors="pt")
    outputs = sentiment_model(**inputs)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1).item()

    # Sınıfların anlamlarını belirleyin (bu örnekte, 0 = olumsuz, 1 = nötr, 2 = olumlu)
    sentiment_map = {0: "olumlu", 1: "nötr", 2: "olumsuz"}
    sentiment = sentiment_map[predicted_class]

    results.append({"entity": entity, "sentiment": sentiment})

# Sonuçları yazdırma
output = {
    "entity_list": entities,
    "results": results
}

print(json.dumps(output, ensure_ascii=False, indent=4))


In [31]:
import re
import pandas as pd
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder, TrigramAssocMeasures, TrigramCollocationFinder
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer, AutoModelForSequenceClassification
import torch
import json

# NLTK veri setlerini indirmek için bağımsız bir blok kullanın
nltk.download('stopwords')
nltk.download('punkt')

def preprocess_text(text):
    # Noktalama işaretlerini kaldırma
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\b(\d{1,2})\s+(Ocak|Şubat|Mart|Nisan|Mayıs|Haziran|Temmuz|Ağustos|Eylül|Ekim|Kasım|Aralık)\s+(\d{4})\b', '', text)
    text = re.sub(r'\b(\d{1,2})/(\d{1,2})/(\d{4})\b', '', text)
    # Küçük harfe dönüştürme
    text = text.lower()

    # Sayıları çıkarma
    text = re.sub(r'\d+', '', text)

    # Kelime tokenizasyonu
    kelimeler = word_tokenize(text)

    # Stop words çıkarma
    stop_words = set(stopwords.words('turkish'))
    kelimeler_filtreli = [kelime for kelime in kelimeler if kelime not in stop_words]

    return kelimeler_filtreli

# NER modeli ve tokenizer yükleme
ner_model = AutoModelForTokenClassification.from_pretrained("savasy/bert-base-turkish-ner-cased")
ner_tokenizer = AutoTokenizer.from_pretrained("savasy/bert-base-turkish-ner-cased")
ner = pipeline('ner', model=ner_model, tokenizer=ner_tokenizer, aggregation_strategy="first")

# Türkçe sentiment analiz modeli ve tokenizer'ı yükleme
sentiment_model = AutoModelForSequenceClassification.from_pretrained("emre/turkish-sentiment-analysis")
sentiment_tokenizer = AutoTokenizer.from_pretrained("emre/turkish-sentiment-analysis")

# Cümleyi analiz etme
sentence = """Fiber 100mb SuperOnline kullanıcısıyım yaklaşık 2 haftadır @Twitch @Kick_Turkey gibi canlı yayın platformlarında 360p yayın izlerken donmalar yaşıyoruz. Başka hiç bir operatörler bu sorunu yaşamazken ben parasını verip alamadığım hizmeti neden ödeyeyim ? @Turkcell"""

# Entity'leri bulma
ner_results = ner(sentence)

# Entity'leri birleştirme ve unique hale getirme
entities = list(set([result['word'] for result in ner_results]))

# Cümleyi ön işleme
processed_sentence = preprocess_text(sentence)

# Tüm metinleri birleştir
all_tokens = processed_sentence

# Bigramları bulma
bigram_measures = BigramAssocMeasures()
bigram_finder = BigramCollocationFinder.from_words(all_tokens)
most_common_bigrams = bigram_finder.ngram_fd.most_common(100)

# Trigramları bulma
trigram_measures = TrigramAssocMeasures()
trigram_finder = TrigramCollocationFinder.from_words(all_tokens)
most_common_trigrams = trigram_finder.ngram_fd.most_common(100)

# Bigram ve trigramları entity listesine ekleme
bigrams_in_sentence = [' '.join(bigram) for bigram, freq in most_common_bigrams if all(word in processed_sentence for word in bigram)]
trigrams_in_sentence = [' '.join(trigram) for trigram, freq in most_common_trigrams if all(word in processed_sentence for word in trigram)]
entities.extend(bigrams_in_sentence)
entities.extend(trigrams_in_sentence)
entities = list(set(entities))  # Unique hale getirme

# Sonuçları tutmak için liste
results = []

# Her entity için analiz yapma
for entity in entities:
    inputs = sentiment_tokenizer(entity, return_tensors="pt")
    outputs = sentiment_model(**inputs)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1).item()

    # Sınıfların anlamlarını belirleyin (bu örnekte, 0 = olumsuz, 1 = nötr, 2 = olumlu)
    sentiment_map = {0: "olumsuz", 1: "nötr", 2: "olumlu"}
    sentiment = sentiment_map[predicted_class]

    results.append({"entity": entity, "sentiment": sentiment})

# Sonuçları yazdırma
output = {
    "entity_list": entities,
    "results": results
}

print(json.dumps(output, ensure_ascii=False, indent=4))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Some weights of the model checkpoint at savasy/bert-base-turkish-ner-cased were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is pa

{
    "entity_list": [
        "Twitch",
        "yaşıyoruz başka",
        "izlerken donmalar yaşıyoruz",
        "superonline kullanıcısıyım",
        "yaşamazken ben",
        "SuperOnline",
        "yayın izlerken",
        "mb superonline kullanıcısıyım",
        "yayın izlerken donmalar",
        "platformlarında p yayın",
        "sorunu yaşamazken ben",
        "kick_turkey canlı yayın",
        "kullanıcısıyım yaklaşık haftadır",
        "Turkcell",
        "yaklaşık haftadır",
        "twitch kick_turkey",
        "yayın platformlarında p",
        "kick_turkey canlı",
        "bir operatörler",
        "yayın platformlarında",
        "haftadır twitch kick_turkey",
        "twitch kick_turkey canlı",
        "haftadır twitch",
        "operatörler sorunu yaşamazken",
        "başka bir",
        "alamadığım hizmeti",
        "izlerken donmalar",
        "ben parasını verip",
        "operatörler sorunu",
        "yaklaşık haftadır twitch",
        "p yayın",
        "hizmeti

In [19]:
import pandas as pd
import json
import xml.etree.ElementTree as ET
from transformers import AutoModelForTokenClassification, AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch

# Türkçe NER modeli ve tokenizer'ı yükleme
ner_model = AutoModelForTokenClassification.from_pretrained("savasy/bert-base-turkish-ner-cased")
ner_tokenizer = AutoTokenizer.from_pretrained("savasy/bert-base-turkish-ner-cased")

# Türkçe sentiment analiz modeli ve tokenizer'ı yükleme
sentiment_model = AutoModelForSequenceClassification.from_pretrained("emre/turkish-sentiment-analysis")
sentiment_tokenizer = AutoTokenizer.from_pretrained("emre/turkish-sentiment-analysis")

# NER pipeline oluşturma
ner = pipeline("ner", model=ner_model, tokenizer=ner_tokenizer, aggregation_strategy="first")

# Sentiment analizi için fonksiyon
def analyze_sentiment(text):
    inputs = sentiment_tokenizer(text, return_tensors="pt")
    outputs = sentiment_model(**inputs)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1).item()

    sentiment_map = {0: "olumsuz", 1: "nötr", 2: "olumlu"}
    return sentiment_map[predicted_class]

# Metni işleme fonksiyonu
def process_text(text):
    ner_results = ner(text)
    entities = list(set([result['word'] for result in ner_results]))

    results = []
    for entity in entities:
        sentiment = analyze_sentiment(entity)
        results.append({"entity": entity, "sentiment": sentiment})

    output = {
        "entity_list": entities,
        "results": results
    }
    return output

# JSON formatında dosya okuma
def read_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

# XML formatında dosya okuma
def read_xml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    data = []
    for elem in root:
        if elem.text and elem.text.strip():  # Check for non-empty text
            data.append(elem.text.strip())
    return data

# CSV formatında dosya okuma
def read_csv(file_path):
    df = pd.read_csv(file_path)
    return df.iloc[:, 0].tolist()  # Assuming text is in the first column

# Dosyayı analiz etme fonksiyonu
def analyze_file(file_path, file_type):
    if file_type == 'json':
        data = read_json(file_path)
    elif file_type == 'xml':
        data = read_xml(file_path)
    elif file_type == 'csv':
        data = read_csv(file_path)
    else:
        raise ValueError("Desteklenmeyen dosya türü. Lütfen 'json', 'xml' veya 'csv' girin.")

    results = []
    for text in data:
        result = process_text(text)
        results.append({"text": text, "analysis": result})

    return results

# Kullanıcıdan metin veya dosya formatı alma
def main():
    input_type = input("Girdi türünü seçin ('text' veya 'file'): ").strip().lower()
    if input_type == 'text':
        input_value = input("Metni girin: ").strip()
        result = process_text(input_value)
        print(json.dumps(result, ensure_ascii=False, indent=4))
    elif input_type == 'file':
        file_path = input("Dosya yolunu girin: ").strip()
        file_type = input("Dosya türünü girin ('json', 'xml', 'csv'): ").strip().lower()
        if file_type not in ['json', 'xml', 'csv']:
            print("Geçersiz dosya türü. Lütfen 'json', 'xml' veya 'csv' girin.")
            return
        results = analyze_file(file_path, file_type)
        print(json.dumps(results, ensure_ascii=False, indent=4))
    else:
        print("Geçersiz giriş türü. Lütfen 'text' veya 'file' girin.")

# Ana fonksiyonu çalıştırma
if __name__ == "__main__":
    main()


Some weights of the model checkpoint at savasy/bert-base-turkish-ner-cased were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Girdi türünü seçin ('text' veya 'file'): text
Metni girin: : Fiber 100mb SuperOnline kullanıcısıyım yaklaşık 2 haftadır @Twitch @Kick_Turkey  gibi canlı yayın platformlarında 360p yayın izlerken donmalar yaşıyoruz. Başka hiç bir operatörler bu sorunu yaşamazken ben parasını verip alamadığım hizmeti neden ödeyeyim ?
{
    "entity_list": [
        "SuperOnline",
        "Fiber",
        "Twitch",
        "Kick _ Turkey"
    ],
    "results": [
        {
            "entity": "SuperOnline",
            "sentiment": "olumlu"
        },
        {
            "entity": "Fiber",
            "sentiment": "olumlu"
        },
        {
            "entity": "Twitch",
            "sentiment": "olumlu"
        },
        {
            "entity": "Kick _ Turkey",
            "sentiment": "olumlu"
        }
    ]
}


In [21]:
import re
import json
import pandas as pd
import xml.etree.ElementTree as ET
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer, AutoModelForSequenceClassification
import torch

def preprocess_text(text):
    # Noktalama işaretlerini kaldırma
    text = re.sub(r'[^\w\s]', '', text)
    # Küçük harfe dönüştürme
    text = text.lower()
    # Sayıları çıkarma
    text = re.sub(r'\d+', '', text)
    return text

# Türkçe NER modeli ve tokenizer'ı yükleme
ner_model = AutoModelForTokenClassification.from_pretrained("savasy/bert-base-turkish-ner-cased")
ner_tokenizer = AutoTokenizer.from_pretrained("savasy/bert-base-turkish-ner-cased")
ner = pipeline("ner", model=ner_model, tokenizer=ner_tokenizer, aggregation_strategy="first")

# Türkçe sentiment analiz modeli ve tokenizer'ı yükleme
sentiment_model = AutoModelForSequenceClassification.from_pretrained("emre/turkish-sentiment-analysis")
sentiment_tokenizer = AutoTokenizer.from_pretrained("emre/turkish-sentiment-analysis")

# Sentiment analizi için fonksiyon
def analyze_sentiment(text):
    inputs = sentiment_tokenizer(text, return_tensors="pt")
    outputs = sentiment_model(**inputs)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1).item()
    sentiment_map = {0: "olumsuz", 1: "nötr", 2: "olumlu"}
    return sentiment_map[predicted_class]

# Metni işleme fonksiyonu
def process_text(text):
    processed_text = preprocess_text(text)
    ner_results = ner(processed_text)
    entities = list(set([result['word'] for result in ner_results]))

    results = []
    for entity in entities:
        sentiment = analyze_sentiment(entity)
        results.append({"entity": entity, "sentiment": sentiment})

    output = {
        "entity_list": entities,
        "results": results
    }
    return output

# JSON formatında dosya okuma
def read_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

# XML formatında dosya okuma
def read_xml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    data = []
    for elem in root:
        if elem.text and elem.text.strip():  # Check for non-empty text
            data.append(elem.text.strip())
    return data

# CSV formatında dosya okuma
def read_csv(file_path):
    df = pd.read_csv(file_path)
    return df.iloc[:, 0].tolist()  # Assuming text is in the first column

# Dosyayı analiz etme fonksiyonu
def analyze_file(file_path, file_type):
    if file_type == 'json':
        data = read_json(file_path)
    elif file_type == 'xml':
        data = read_xml(file_path)
    elif file_type == 'csv':
        data = read_csv(file_path)
    else:
        raise ValueError("Desteklenmeyen dosya türü. Lütfen 'json', 'xml' veya 'csv' girin.")

    results = []
    for text in data:
        result = process_text(text)
        results.append({"text": text, "analysis": result})

    return results

# Kullanıcıdan metin veya dosya formatı alma
def main():
    input_type = input("Girdi türünü seçin ('text' veya 'file'): ").strip().lower()
    if input_type == 'text':
        input_value = input("Metni girin: ").strip()
        result = process_text(input_value)
        print(json.dumps(result, ensure_ascii=False, indent=4))
    elif input_type == 'file':
        file_path = input("Dosya yolunu girin: ").strip()
        file_type = input("Dosya türünü girin ('json', 'xml', 'csv'): ").strip().lower()
        if file_type not in ['json', 'xml', 'csv']:
            print("Geçersiz dosya türü. Lütfen 'json', 'xml' veya 'csv' girin.")
            return
        results = analyze_file(file_path, file_type)
        print(json.dumps(results, ensure_ascii=False, indent=4))
    else:
        print("Geçersiz giriş türü. Lütfen 'text' veya 'file' girin.")

# Ana fonksiyonu çalıştırma
if __name__ == "__main__":
    main()


Some weights of the model checkpoint at savasy/bert-base-turkish-ner-cased were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Girdi türünü seçin ('text' veya 'file'): text
Metni girin: @TurkcellHizmet ya vereceğiniz hizmeti... ya bilader allahın adasında bile iyi çekerken, şehir merkezinde nasıl çekmiyor bu 
{
    "entity_list": [
        "turkcellhizmet"
    ],
    "results": [
        {
            "entity": "turkcellhizmet",
            "sentiment": "olumlu"
        }
    ]
}


In [ ]:
import re
import pandas as pd
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder, TrigramAssocMeasures, TrigramCollocationFinder
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer

# NLTK veri setlerini indirmek için bağımsız bir blok kullanın
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')

def preprocess_text(text):
    # Noktalama işaretlerini kaldırma
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\b(\d{1,2})\s+(Ocak|Şubat|Mart|Nisan|Mayıs|Haziran|Temmuz|Ağustos|Eylül|Ekim|Kasım|Aralık)\s+(\d{4})\b', '', text)
    text = re.sub(r'\b(\d{1,2})/(\d{1,2})/(\d{4})\b', '', text)
    # Küçük harfe dönüştürme
    text = text.lower()

    # Sayıları çıkarma
    text = re.sub(r'\d+', '', text)

    # Kelime tokenizasyonu
    kelimeler = word_tokenize(text)

    # Stop words çıkarma
    stop_words = set(stopwords.words('turkish'))
    kelimeler_filtreli = [kelime for kelime in kelimeler if kelime not in stop_words]

    return kelimeler_filtreli

def filter_words_by_pos(words, pos_tags):
    pos_tagged_words = nltk.pos_tag(words, tagset='universal')
    filtered_words = [word for word, pos in pos_tagged_words if pos in pos_tags]
    return filtered_words

# NER modeli ve tokenizer yükleme
model = AutoModelForTokenClassification.from_pretrained("savasy/bert-base-turkish-ner-cased")
tokenizer = AutoTokenizer.from_pretrained("savasy/bert-base-turkish-ner-cased")
ner = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="first")

# Örnek veri
data = [
    "Fiber 100mb SuperOnline kullanıcısıyım yaklaşık 2 haftadır @Twitch @Kick_Turkey gibi canlı yayın platformlarında 360p yayın izlerken donmalar yaşıyoruz. Başka hiç bir operatörler bu sorunu yaşamazken ben parasını verip alamadığım hizmeti neden ödeyeyim ? @Turkcell"
]

# Veri ön işleme
processed_data = [preprocess_text(comment) for comment in data]

# Tüm metinleri birleştir
all_tokens = [token for sublist in processed_data for token in sublist]

# Tekil kelimeleri bulma
word_freq = Counter(all_tokens)
most_common_words = [word for word, freq in word_freq.most_common(5)]  # İlk 50 kelimeyi alalım

# POS etiketleri ile filtreleme: fiiller (VERB), sıfatlar (ADJ), isimler (NOUN)
filtered_words = filter_words_by_pos(most_common_words, ['NOUN', 'PRONOUN'])

# Bigramları bulma
bigram_measures = BigramAssocMeasures()
bigram_finder = BigramCollocationFinder.from_words(all_tokens)
most_common_bigrams = bigram_finder.ngram_fd.most_common(10)

# Trigramları bulma
trigram_measures = TrigramAssocMeasures()
trigram_finder = TrigramCollocationFinder.from_words(all_tokens)
most_common_trigrams = trigram_finder.ngram_fd.most_common(10)

# Boş bir DataFrame oluştur
df = pd.DataFrame(columns=["comment", "entity_list"])

for comment in data:
    # Yorumları ön işleme
    processed_comment = preprocess_text(comment)

    # NER ile entity tespiti
    entities = ner(comment)
    org_entities = [entity['word'] for entity in entities if entity['entity_group'] == 'ORG']

    # En sık geçen kelimeler, bigramlar ve trigramlar
    words_in_comment = [word for word in filtered_words if word in processed_comment]
    bigrams_in_comment = [' '.join(bigram) for bigram, freq in most_common_bigrams if all(word in processed_comment for word in bigram)]
    trigrams_in_comment = [' '.join(trigram) for trigram, freq in most_common_trigrams if all(word in processed_comment for word in trigram)]

    # Entity listesine ekleme
    all_entities = list(set(org_entities + words_in_comment + bigrams_in_comment + trigrams_in_comment))

    # Yeni satırı DataFrame'e ekle
    new_row = {"comment": comment, "entity_list": all_entities}
    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

# Entity listesi ile birlikte DataFrame'i döndürme
import json
output = df.to_dict(orient="records")
print(json.dumps(output, ensure_ascii=False, indent=4))


In [39]:
df

,comment,entity_list
0,Fiber 100mb SuperOnline kullanıcısıyım yaklaşı...,"[Twitch, yaşıyoruz başka, izlerken donmalar ya..."


In [ ]:
df.iloc[2000]['comment']

'Turkcell faturasız paketler her hafta pahalanıyor. Bir bakıyorum yine artmış hiç düşmüyor fiyatlar hep yükseliyor. Şebeke çekmiyor internet paketleri aldı başını gitti. Böyle giderse operatör değiştireceğim. Bu nedir ya. Bir kere de şaşırt bizi Turkcell (! )'

In [ ]:
df.iloc[2000]['entity_list']

['bir', 'böyle bir', 'internet', 'operatör', 'bir operatör', 'Turkcell']

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, load_metric
import numpy as np

# Model ve tokenizer'ı yükle
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# Veri setini yükle (örnek: veri setinizi buraya yükleyin)
dataset = load_dataset("path/to/your/dataset")

# Tokenizasyon
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Eğitim ve test veri setlerini ayırma
train_dataset = tokenized_datasets["train"]
test_dataset = tokenized_datasets["test"]

# Eğitim argümanlarını tanımlama
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Trainer'ı tanımlama
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Eğitimi başlatma
trainer.train()

# Modeli değerlendirme
results = trainer.evaluate()
print(results)


In [42]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig, pipeline

config_sentiment = AutoConfig.from_pretrained("saribasmetehan/bert-base-turkish-sentiment-analysis")
model_sentiment = AutoModelForSequenceClassification.from_pretrained("saribasmetehan/bert-base-turkish-sentiment-analysis", config=config_sentiment)
tokenizer_sentiment = AutoTokenizer.from_pretrained("saribasmetehan/bert-base-turkish-sentiment-analysis")


device = torch.device('cpu') #or 'cuda'

sentiment_pipeline = pipeline(
    task='text-classification',
    model=model_sentiment,
    tokenizer=tokenizer_sentiment,
    framework='pt',
    #device=device
)

input = """ Fiber 100mb SuperOnline kullanıcısıyım yaklaşık 2 haftadır
@Twitch
@Kick_Turkey
 gibi canlı yayın platformlarında 360p yayın izlerken donmalar
yaşıyoruz
"""

sentiment_result = sentiment_pipeline(input)
print(sentiment_result)


config.json:   0%|          | 0.00/858 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/251k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/755k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'LABEL_1', 'score': 0.8027313947677612}]
